In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import math

In [2]:
class PoseDataset(Dataset):    
    def __init__(self, data_dir = None):
        self._data_dir = data_dir
        data_dirs = [x[0] for x in os.walk(self._data_dir)][1:]
        annotations = []
        for i in range(0,len(data_dirs)):
            annotation = json.load(open(f"{data_dirs[i]}/annotations.json"))['annotations']
            annotations.extend(annotation)
        self._annotations_person = [ano for ano in annotations if ano['category_id'] == 0]
        self._pose_dict = {"liftingbad": 0, 
                           "liftinggood": 1, 
                           "reachingbad": 2, 
                           "reachinggood": 3, 
                           'randomrandom': 4}

    def _process_pose(self, pose):
        return self._pose_dict[pose]
    
    def _get_pose_size(self, keypoints, ratio):
        hips_center = (keypoints[9,:] + keypoints[10,:]) / 2
        shoulders_center = (keypoints[3,:] + keypoints[4,:]) / 2
        torso_size = np.linalg.norm((shoulders_center - hips_center))
        distance = np.linalg.norm((keypoints - hips_center), axis = 1)
        max_d = np.max(distance)
        pose_size = max(torso_size * ratio, max_d)
        return pose_size
    def _normalize_pose(self,keypoints):
        data_p = np.expand_dims(np.array(keypoints), axis=1).reshape(-1,3)[:,0:2]
        data_p = np.delete(data_p,[3,4], axis = 0)
        hip_center = (data_p[9,:] + data_p[10,:]) / 2
        data_p = data_p - hip_center
        pose_size = self._get_pose_size(data_p, 2)
        data_p = data_p / pose_size
        return data_p.flatten()
    def __getitem__(self, idx):
        actor = self._annotations_person[idx]
        x = torch.tensor(self._normalize_pose(actor['keypoints']))
        pose = actor['pose_category'] + actor['pose_subcategory']
        y = self._process_pose(pose)
        return x, y
    def __len__(self):
        return len(self._annotations_person)

In [3]:
pose_data = PoseDataset(data_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/')

In [4]:
num_val= int(len(pose_data) * 0.1)
num_train = len(pose_data) - 2 * num_val
train, val, test = random_split(pose_data, [num_train, num_val, num_val])
batch_size = 128
dataloaders = {
    'train': DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8),
    'val': DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8),
}
dataset_sizes = {}
dataset_sizes['train'] = len(train)
dataset_sizes['val'] = len(val)
dataset_sizes['test'] = len(test)
print(dataset_sizes)

{'train': 47915, 'val': 5989, 'test': 5989}


In [5]:
print(f"number of datapoints: {len(pose_data)}")

number of datapoints: 59893


In [6]:
dataloader_test = DataLoader(pose_data, batch_size=1, shuffle=False)

In [7]:
x, y = next(iter(dataloader_test))
print(f"size of one batch input data {x.shape}")
print(f"size of one batch label data {y.shape}")


size of one batch input data torch.Size([1, 30])
size of one batch label data torch.Size([1])


In [9]:
from torch import nn
class PoseModel(nn.Module):
    
    def __init__(self, input_size, num_classes):
        super(PoseModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.softmax(x)
        return x

In [14]:
pose_model = PoseModel(30, 5)
out = pose_model(x.float())
print(out)

tensor([[0.1871, 0.2299, 0.2041, 0.1853, 0.1936]], grad_fn=<SoftmaxBackward>)


In [11]:
loss = nn.CrossEntropyLoss()
print(out)
print(y)
loss(out, y)

tensor([[0.1747, 0.2007, 0.1961, 0.2019, 0.2265]], grad_fn=<SoftmaxBackward>)
tensor([3])


tensor(1.6077, grad_fn=<NllLossBackward>)

In [12]:
from torch import nn
import wandb
from torch.optim import lr_scheduler
import time
import copy
from datetime import datetime


def train_model(model, dataloaders, dataset_sizes, optimizer, config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss()
    wandb.init(project =  "ergo_ml_training", entity = "voxel-wandb", config = config, tags = [config['tags']])
    scheduler = lr_scheduler.StepLR(optimizer, step_size=config['step'], gamma=0.1)
    config = wandb.config
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())   
    best_accuracy = 0
    model.to(device)
    today_date = datetime.today().strftime('%Y-%m-%d')
    for epoch in range(config.num_epochs):
        print('Epoch {}/{}'.format(epoch, config.num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase] 
            if phase == 'train':
                scheduler.step()
                
            if phase == 'val':
                #print("val accuracy", epoch_acc)
                wandb.log({"val loss":epoch_loss})
                wandb.log({"val_accuracy":epoch_acc})
            if phase == 'train':
                #print("train accuracy", epoch_acc)
                #print("train loss", epoch_loss)
                wandb.log({"train loss":epoch_loss})
                wandb.log({"train_accuracy":epoch_acc})
            print(f'{phase} Loss: {epoch_loss} Acc: {epoch_acc}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_accuracy))

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f"voxel_ergo_ml_{config.tags}_{today_date}.pth")
    wandb.join()
    return model
    

In [15]:
config = {
        'num_epochs': 100,
        'tags': "ergoMLLinear",
        'step': 70,
}

optimizer = torch.optim.SGD(pose_model.parameters(), lr=0.1, momentum=0.9)
train_model(pose_model, dataloaders, dataset_sizes, optimizer, config)

train loss,█▅▄▄▄▄▄▄▄▄▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▄▅▅▅▅▅▅▅▅▅▅▅▆▇▇▇▇▇█████████████████████
val loss,█▇▇▆▆▆▆▆▆▆▆▆▆▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▂▂▂▃▃▃▃▃▃▃▃▃▆▇▇▇▇▇▇████████████████████
train loss,1.003
train_accuracy,0.90101
val loss,0.97461
val_accuracy,0.92887


wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-05-05 18:26:55.092141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 18:26:55.092190: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 0/99
----------
train Loss: 1.3764709845572465 Acc: 0.5295418971094648
val Loss: 1.1992964968495823 Acc: 0.7144765403239273
Epoch 1/99
----------
train Loss: 1.2002458540343124 Acc: 0.7116977981842847
val Loss: 1.1626792425583035 Acc: 0.7436967774252797
Epoch 2/99
----------
train Loss: 1.1780852417128327 Acc: 0.7295836376917458
val Loss: 1.1532839266161903 Acc: 0.7503756887627318
Epoch 3/99
----------
train Loss: 1.170954509370078 Acc: 0.7353020974642597
val Loss: 1.147275271012043 Acc: 0.757722491233929
Epoch 4/99
----------
train Loss: 1.1645185865490593 Acc: 0.7415005739330064
val Loss: 1.1449737182183424 Acc: 0.7597261646351645
Epoch 5/99
----------
train Loss: 1.1604076843402038 Acc: 0.7456746321611187
val Loss: 1.1474575130743223 Acc: 0.75722157288362
Epoch 6/99
----------
train Loss: 1.1586881587864348 Acc: 0.7465929249713034
val Loss: 1.1405852168803423 Acc: 0.7635665386541994
Epoch 7/99
----------
train Loss: 1.1548219775924675 Acc: 0.7505791505791506
val Loss: 1.138300

train Loss: 1.002199240496123 Acc: 0.9024939997912972
val Loss: 0.9693024490388771 Acc: 0.9352145600267158
Epoch 64/99
----------
train Loss: 0.9990077563694545 Acc: 0.9049775644370239
val Loss: 0.9740540104988004 Acc: 0.9298714309567542
Epoch 65/99
----------
train Loss: 0.9986594621764212 Acc: 0.9057706355003653
val Loss: 0.9722453310306929 Acc: 0.9327099682751713
Epoch 66/99
----------
train Loss: 1.0013810530642335 Acc: 0.9036209955128874
val Loss: 0.9841592531905665 Acc: 0.9200200367340124
Epoch 67/99
----------
train Loss: 0.999327457551977 Acc: 0.9048732129813212
val Loss: 0.976804487393523 Acc: 0.9275338119886459
Epoch 68/99
----------
train Loss: 0.9986969234686387 Acc: 0.9054158405509758
val Loss: 0.9779517444889102 Acc: 0.9263650025045919
Epoch 69/99
----------
train Loss: 0.9976940328164525 Acc: 0.9067932797662528
val Loss: 0.9708963658957355 Acc: 0.9335448321923527
Epoch 70/99
----------
train Loss: 0.9877422195802983 Acc: 0.9168527600960034
val Loss: 0.9663895168334699 Ac

train loss,█▅▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▄▅▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████████
val loss,█▇▆▆▆▆▆▆▆▆▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▂▃▃▃▃▃▃▃▆▇▇▇▇▇▇▇▇▇▇▇▇███▇▇████████████
train loss,0.97274
train_accuracy,0.932
val loss,0.95879
val_accuracy,0.9454


PoseModel(
  (layer1): Linear(in_features=30, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=5, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)

In [16]:
model = PoseModel(30, 5)
model.load_state_dict(
    torch.load('/home/reza_voxelsafety_com/voxel/experimental/reza/Ergonomic/voxel_ergo_ml_ergoMLLinear_2022-05-05.pth')
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloaders['test'] = DataLoader(val, batch_size=1, shuffle=False, num_workers=8)
running_corrects = 0
model.to(device)
for inputs, labels in dataloaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs.float())
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == labels.data)
accuracy_test = running_corrects.double() / dataset_sizes['test']
print(f"Accuracy test {accuracy_test}")

Accuracy test 0.9268659208549007
